In [1]:
import warnings
import csv
import pandas as pd
import numpy as np

In [2]:
warnings.filterwarnings('ignore')

Read excel and drop the empty columns

In [3]:
df = pd.read_excel('NAC LIST 2021 2 Year Follow Up Status.xlsx', sheet_name = 'Sheet2')
df = df.dropna(axis = 1, how = 'all') 
df

,Study ID,Age,Diagnosis,Stage,Bx date DD/MM/YYYY,Last Follow up date or date of death of died,Vital Status,HIV status,CD4 at cancer dignosis,VL at cancer diangosis,Dose of chemo,Cycles of chemo,Chemo regimen,Path response at surgery,Unnamed: 15,Unnamed: 16
0,5285,33,CACX,IB1,19-01-2017,24-07-2020,Dead,Negative,NaN,NaN,230mg,3,Carbo/paclitaxol,"No surgery, CRT",NaN,NaN
1,5290,42,CACX,IB1,2016-06-05 00:00:00,17-03-2021,Alive,Negative,NaN,NaN,90mg,3,Cisplatin/paclitaxel,TAH/BSO but no path in IPMS,CR,No nodes
2,5373,53,CACX,IB1,2017-04-04 00:00:00,2021-03-08 00:00:00,Alive,Positive,505 L,<400,350mg,3,carbo/paclitaxel,TAH/BSO CIN 3 Clear margins (17:PS003853R),CR,No nodes
3,5397,58,CACX,IA2,2017-05-09 00:00:00,18-11-2017,Dead,Positive,376 L,<400,290mg,3,Carbo/paclitaxol,Right/left pelvic lymph node:Reactive hypeplas...,CR,No nodes
4,5525,48,CACX,IB1,2017-04-09 00:00:00,2021-02-17 00:00:00,Alive,Positive,371 L,<400,30mg,2,neupogen,Surgery at GPH so no path result in ipms,IB1-IA2,No nodes
5,6144,59,CACX,IB1,2017-07-07 00:00:00,2021-03-08 00:00:00,Alive,Positive,127 *L,<400,60mg,2,cisplatin at GPH,No surgery (U54) CRT,NaN,NaN
6,5508,43,CACX,IB2,28-08-2017,2021-01-13 00:00:00,Alive,Positive,481 L,<400,660mg,3,Carboplatin/Doce,No result in IPMS but she did TAH at SLH,IB2-IA1,No nodes
7,5302,47,CACX,IB1,2017-01-09 00:00:00,2021-03-08 00:00:00,Alive,Positive,31 *L,<400,790mg,3,Carboplatin/Doce,She did TAH at GPH,IB1-IA2,No nodes
8,5615,63,CACX,IB1,27-11-2017,2021-03-09 00:00:00,Alive,Negative,NaN,NaN,None,0,NaN,No result in IPMS,IB1,No nodes
9,5745,52,CACX,IB1,28-06-2018,2021-02-17 00:00:00,Alive,Positive,None in ipms,None in ipms,unknown,3,pt not in the 2018 register,NSurgery at GPH. No result in ipms,NaN,NaN


In [4]:
df.columns.values

array(['Study ID', 'Age', 'Diagnosis', 'Stage', 'Bx date DD/MM/YYYY',
       'Last Follow up date or date of death of died', 'Vital Status ',
       'HIV status ', 'CD4 at cancer dignosis ',
       'VL at cancer diangosis ', 'Dose of chemo', 'Cycles of chemo',
       'Chemo regimen ', 'Path response at surgery ', 'Unnamed: 15',
       'Unnamed: 16'], dtype=object)

Fix spelling errors in columns headers

In [5]:
df['HIV status'] = df['HIV status ']
df['Vital status'] = df['Vital Status ']
df['VL at cancer diagnosis'] = df['VL at cancer diangosis ']
df['CD4 at cancer diagnosis'] = df['CD4 at cancer dignosis ']
df['Chemo regimen'] = df['Chemo regimen ']
df['Path response at surgery'] = df['Path response at surgery ']
df = df.drop(columns = ['HIV status ', 'Vital Status ', 'VL at cancer diangosis ', 'CD4 at cancer dignosis ', 'Chemo regimen ', 'Path response at surgery '])
df

,Study ID,Age,Diagnosis,Stage,Bx date DD/MM/YYYY,Last Follow up date or date of death of died,Dose of chemo,Cycles of chemo,Unnamed: 15,Unnamed: 16,HIV status,Vital status,VL at cancer diagnosis,CD4 at cancer diagnosis,Chemo regimen,Path response at surgery
0,5285,33,CACX,IB1,19-01-2017,24-07-2020,230mg,3,NaN,NaN,Negative,Dead,NaN,NaN,Carbo/paclitaxol,"No surgery, CRT"
1,5290,42,CACX,IB1,2016-06-05 00:00:00,17-03-2021,90mg,3,CR,No nodes,Negative,Alive,NaN,NaN,Cisplatin/paclitaxel,TAH/BSO but no path in IPMS
2,5373,53,CACX,IB1,2017-04-04 00:00:00,2021-03-08 00:00:00,350mg,3,CR,No nodes,Positive,Alive,<400,505 L,carbo/paclitaxel,TAH/BSO CIN 3 Clear margins (17:PS003853R)
3,5397,58,CACX,IA2,2017-05-09 00:00:00,18-11-2017,290mg,3,CR,No nodes,Positive,Dead,<400,376 L,Carbo/paclitaxol,Right/left pelvic lymph node:Reactive hypeplas...
4,5525,48,CACX,IB1,2017-04-09 00:00:00,2021-02-17 00:00:00,30mg,2,IB1-IA2,No nodes,Positive,Alive,<400,371 L,neupogen,Surgery at GPH so no path result in ipms
5,6144,59,CACX,IB1,2017-07-07 00:00:00,2021-03-08 00:00:00,60mg,2,NaN,NaN,Positive,Alive,<400,127 *L,cisplatin at GPH,No surgery (U54) CRT
6,5508,43,CACX,IB2,28-08-2017,2021-01-13 00:00:00,660mg,3,IB2-IA1,No nodes,Positive,Alive,<400,481 L,Carboplatin/Doce,No result in IPMS but she did TAH at SLH
7,5302,47,CACX,IB1,2017-01-09 00:00:00,2021-03-08 00:00:00,790mg,3,IB1-IA2,No nodes,Positive,Alive,<400,31 *L,Carboplatin/Doce,She did TAH at GPH
8,5615,63,CACX,IB1,27-11-2017,2021-03-09 00:00:00,None,0,IB1,No nodes,Negative,Alive,NaN,NaN,NaN,No result in IPMS
9,5745,52,CACX,IB1,28-06-2018,2021-02-17 00:00:00,unknown,3,NaN,NaN,Positive,Alive,None in ipms,None in ipms,pt not in the 2018 register,NSurgery at GPH. No result in ipms


Present outcomes with this treatment protocol:
    - Age
    - Marital 
    - Chemo and surgery
    - Treatment response
    - Followup time
    - Vital status
    - HIV status
    - CD4 at cancer diagnosis
    - VL at cancer diagnosis

In [6]:
# Do we need diagnosis? 
data = df[['Study ID', 'Vital status', 'HIV status', 'Age', 'Stage', 'Last Follow up date or date of death of died', 'VL at cancer diagnosis', 'CD4 at cancer diagnosis', 'Chemo regimen', 'Dose of chemo', 'Cycles of chemo']]
data

,Study ID,Vital status,HIV status,Age,Stage,Last Follow up date or date of death of died,VL at cancer diagnosis,CD4 at cancer diagnosis,Chemo regimen,Dose of chemo,Cycles of chemo
0,5285,Dead,Negative,33,IB1,24-07-2020,NaN,NaN,Carbo/paclitaxol,230mg,3
1,5290,Alive,Negative,42,IB1,17-03-2021,NaN,NaN,Cisplatin/paclitaxel,90mg,3
2,5373,Alive,Positive,53,IB1,2021-03-08 00:00:00,<400,505 L,carbo/paclitaxel,350mg,3
3,5397,Dead,Positive,58,IA2,18-11-2017,<400,376 L,Carbo/paclitaxol,290mg,3
4,5525,Alive,Positive,48,IB1,2021-02-17 00:00:00,<400,371 L,neupogen,30mg,2
5,6144,Alive,Positive,59,IB1,2021-03-08 00:00:00,<400,127 *L,cisplatin at GPH,60mg,2
6,5508,Alive,Positive,43,IB2,2021-01-13 00:00:00,<400,481 L,Carboplatin/Doce,660mg,3
7,5302,Alive,Positive,47,IB1,2021-03-08 00:00:00,<400,31 *L,Carboplatin/Doce,790mg,3
8,5615,Alive,Negative,63,IB1,2021-03-09 00:00:00,NaN,NaN,NaN,None,0
9,5745,Alive,Positive,52,IB1,2021-02-17 00:00:00,None in ipms,None in ipms,pt not in the 2018 register,unknown,3


Changing categorial variables to boolean values

In [7]:
# For vital status, alive is 0, dead is 1
data.loc[df['Vital status'] == 'Dead', 'Vital status'] = 1
data.loc[df['Vital status'] == 'Alive', 'Vital status'] = 0
data.loc[df['Vital status'] == 'Alive ', 'Vital status'] = 0
# for HIV status, positive is 0, negative is 1
data.loc[df['HIV status'] == 'Negative', 'HIV status'] = 1
data.loc[df['HIV status'] == 'Positive', 'HIV status'] = 0

Changing CD4 values to floats

In [8]:
data['CD4 at cancer diagnosis'] = data['CD4 at cancer diagnosis'].replace('None in ipms', np.nan) 
data['CD4 at cancer diagnosis'] = data['CD4 at cancer diagnosis'].str.strip(' *L').astype(float)

Changing VL values to boolean values

In [9]:
data['VL at cancer diagnosis'] = data['VL at cancer diagnosis'].replace('None in ipms', np.nan) 
# VL of <400 is 1, NaN 
data.loc[df['VL at cancer diagnosis'] == '<400', 'VL at cancer diagnosis'] = 1

In [10]:
data.head(12)

,Study ID,Vital status,HIV status,Age,Stage,Last Follow up date or date of death of died,VL at cancer diagnosis,CD4 at cancer diagnosis,Chemo regimen,Dose of chemo,Cycles of chemo
0,5285,1,1,33,IB1,24-07-2020,NaN,NaN,Carbo/paclitaxol,230mg,3
1,5290,0,1,42,IB1,17-03-2021,NaN,NaN,Cisplatin/paclitaxel,90mg,3
2,5373,0,0,53,IB1,2021-03-08 00:00:00,1,505.0,carbo/paclitaxel,350mg,3
3,5397,1,0,58,IA2,18-11-2017,1,376.0,Carbo/paclitaxol,290mg,3
4,5525,0,0,48,IB1,2021-02-17 00:00:00,1,371.0,neupogen,30mg,2
5,6144,0,0,59,IB1,2021-03-08 00:00:00,1,127.0,cisplatin at GPH,60mg,2
6,5508,0,0,43,IB2,2021-01-13 00:00:00,1,481.0,Carboplatin/Doce,660mg,3
7,5302,0,0,47,IB1,2021-03-08 00:00:00,1,31.0,Carboplatin/Doce,790mg,3
8,5615,0,1,63,IB1,2021-03-09 00:00:00,NaN,NaN,NaN,None,0
9,5745,0,0,52,IB1,2021-02-17 00:00:00,NaN,NaN,pt not in the 2018 register,unknown,3


Converting dates to datetime values

In [11]:
data['Last Follow up date or date of death of died'] = pd.to_datetime(data['Last Follow up date or date of death of died'])

Fixing irregular values of 'unknown' and 'None' to NaN

In [12]:
data['Dose of chemo'] = data['Dose of chemo'].replace('unknown', np.nan).replace('None', np.nan)

Add enrollment dates to data

In [13]:
all_patients = pd.read_csv("patient_enrollment_dates.csv")
data = data.merge(all_patients[["patient_id", "enroll_date"]], how = "left", left_on = "Study ID", right_on = "patient_id").drop(columns = "patient_id")

Finalize data

In [14]:
data = data.head(10)
data

,Study ID,Vital status,HIV status,Age,Stage,Last Follow up date or date of death of died,VL at cancer diagnosis,CD4 at cancer diagnosis,Chemo regimen,Dose of chemo,Cycles of chemo,enroll_date
0,5285,1,1,33,IB1,2020-07-24,NaN,NaN,Carbo/paclitaxol,230mg,3,2017-03-01
1,5290,0,1,42,IB1,2021-03-17,NaN,NaN,Cisplatin/paclitaxel,90mg,3,2017-03-08
2,5373,0,0,53,IB1,2021-03-08,1,505.0,carbo/paclitaxel,350mg,3,2017-05-17
3,5397,1,0,58,IA2,2017-11-18,1,376.0,Carbo/paclitaxol,290mg,3,2017-05-31
4,5525,0,0,48,IB1,2021-02-17,1,371.0,neupogen,30mg,2,2017-09-13
5,6144,0,0,59,IB1,2021-03-08,1,127.0,cisplatin at GPH,60mg,2,2019-06-06
6,5508,0,0,43,IB2,2021-01-13,1,481.0,Carboplatin/Doce,660mg,3,2017-08-30
7,5302,0,0,47,IB1,2021-03-08,1,31.0,Carboplatin/Doce,790mg,3,2017-03-15
8,5615,0,1,63,IB1,2021-03-09,NaN,NaN,NaN,NaN,0,2017-12-06
9,5745,0,0,52,IB1,2021-02-17,NaN,NaN,pt not in the 2018 register,NaN,3,2018-05-09


Describe the columns in the data

In [15]:
data['Vital status'].describe()

count     10
unique     2
top        0
freq       8
Name: Vital status, dtype: int64

In [16]:
data['HIV status'].describe()

count     10
unique     2
top        0
freq       7
Name: HIV status, dtype: int64

In [17]:
data['Age'].describe()

count    10.000000
mean     49.800000
std       9.077445
min      33.000000
25%      44.000000
50%      50.000000
75%      56.750000
max      63.000000
Name: Age, dtype: float64

In [18]:
data['Stage'].describe()

count      10
unique      3
top       IB1
freq        8
Name: Stage, dtype: object

In [19]:
data.groupby(data['Stage'])['Study ID'].count() 

Stage
IA2    1
IB1    8
IB2    1
Name: Study ID, dtype: int64

In [20]:
data['Last Follow up date or date of death of died'].describe()

count                      10
unique                      7
top       2021-03-08 00:00:00
freq                        3
first     2017-11-18 00:00:00
last      2021-03-17 00:00:00
Name: Last Follow up date or date of death of died, dtype: object

In [21]:
data['VL at cancer diagnosis'].describe()

count     6
unique    1
top       1
freq      6
Name: VL at cancer diagnosis, dtype: int64

In [22]:
data['CD4 at cancer diagnosis'].describe()

count      6.000000
mean     315.166667
std      193.138724
min       31.000000
25%      188.000000
50%      373.500000
75%      454.750000
max      505.000000
Name: CD4 at cancer diagnosis, dtype: float64

In [23]:
data['Chemo regimen'].describe()

count                    9
unique                   7
top       Carbo/paclitaxol
freq                     2
Name: Chemo regimen, dtype: object

In [24]:
data['Dose of chemo'].describe()

count         8
unique        8
top       350mg
freq          1
Name: Dose of chemo, dtype: object

In [25]:
data['Cycles of chemo'].describe()

count    10.000000
mean      2.500000
std       0.971825
min       0.000000
25%       2.250000
50%       3.000000
75%       3.000000
max       3.000000
Name: Cycles of chemo, dtype: float64

In [26]:
data.to_csv('article_4_cleaned_data.csv')